by wedad\
Hey team this file contains the optimisation for logistic Regression Optimzation and experimentations\
I'll be comparing them to the baseline model found in baseline_model.ipynb\
The columns dropped are selected from '4_FeatureEngineeringRegression' file

# LogisticRegression Optimisation
Approaches I took were feature selection, Hyperparam tuning, Different logistic Model testing\
And Ensemble of different models\
Overall the best performing model was the Ensemble with reduced feature set, as see below:
--------------------------------------------



Read in all the relevant Libraries


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
import warnings
warnings.filterwarnings('ignore')

In [2]:
seed_list = [1,2,3,4,5,6,7,8,9,10]

Read In the data

In [3]:
aki_df = pd.read_csv('../data/df_final_AKI.csv')
aki_df = aki_df.drop(columns=['Unnamed: 0', 'subject_id'])
aki_df

,dod,gender,age,Albumin,Creatinine,Hemoglobin,INR(PT),PT,Sodium,Urea Nitrogen,Arterial Blood Pressure diastolic,Arterial Blood Pressure systolic,Heart Rate,Respiratory Rate,hypertension,chronic_kidney_disease,sepsis,Intercept
0,0,1,73,3.6,1.2,9.4,3.2,13.8,138.0,16.0,48.0,101.0,84.0,20.0,1,0,0,1
1,0,0,24,4.1,0.5,10.2,1.2,15.7,143.0,6.0,56.0,114.0,59.0,16.0,0,0,0,1
2,0,0,54,4.1,0.4,10.2,1.0,14.4,142.0,22.0,48.0,130.0,59.0,24.0,1,0,0,1
3,0,1,84,3.5,1.1,10.2,1.6,18.7,144.0,17.0,56.0,114.0,89.0,21.0,1,0,0,1
4,0,0,59,3.6,0.6,10.2,1.7,12.4,153.0,8.0,70.0,91.0,99.0,21.0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804,0,1,61,3.3,3.6,10.2,1.5,13.1,138.0,23.0,62.0,103.0,89.0,20.0,0,1,0,1
9805,0,1,74,3.5,0.6,10.2,1.2,13.6,138.0,6.0,56.0,114.0,107.0,22.0,1,0,0,1
9806,0,1,58,4.1,0.6,10.2,0.9,9.3,141.0,8.0,56.0,114.0,78.0,10.0,0,0,0,1
9807,1,1,84,4.0,3.4,12.2,1.1,17.9,141.0,88.0,56.0,114.0,101.0,25.0,1,1,0,1


In [4]:
model_desc = []
model_comparison_DF = pd.DataFrame(model_desc, columns=['model', 'accuracy', 'precision', 'recall', 'F1 score'])
model_comparison_DF

,model,accuracy,precision,recall,F1 score


The function below takes in different Linear Models, and tests them over number of different Seeds to get an Average perfromance

In [6]:
x = aki_df.drop(columns=['dod'])
y = aki_df['dod']

def logisticTester(model_desc, model, x, y, title='alg'):
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    for rand in seed_list:
        x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=rand)
        model.fit(x_train, y_train)
        y_pred_log = model.predict(x_test)
        total_acc += accuracy_score(y_test, y_pred_log)
        total_pre += precision_score(y_test, y_pred_log)
        total_rec += recall_score(y_test, y_pred_log)
        total_f1 += f1_score(y_test, y_pred_log)

    model_desc.append([
        str(model),
        total_acc/total,
        total_pre/total,
        total_rec/total,
        total_f1/total
        
                      ])
    # Evaluate
    print(f"{title}")
    print('--------------------------------------------')
    print("Accuracy:", total_acc/total,)
    print("Precision: ", total_pre/total,)
    print("recall: ", total_rec/total,)
    print("f1_score: ", total_f1/total,)
    print('--------------------------------------------')
    # print("Classification Report:\n", classification_report(y_test, y_pred_bin))
    plt.show()
    

In [7]:
log_reg = LogisticRegression()
logisticTester(model_desc, log_reg, x, y,'Linear Regression baseline')

Linear Regression baseline
--------------------------------------------
Accuracy: 0.8441092539747249
Precision:  0.6553213679545706
recall:  0.1669879518072289
f1_score:  0.2658264709678952
--------------------------------------------


Baseline model, accuracy of approximately 0.844\
It is important that due to the imbalance\ 
Predicting every value to be 0 gives 83% accuracy\
In our case, it is important that we focus on Precision and Recall

In [8]:
x = aki_df.drop(columns=['dod','Sodium','Hemoglobin','gender','hypertension','Intercept'])
y = aki_df['dod']
x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=seed_list[0])

In [9]:
logisticTester(model_desc, log_reg, x, y,'Linear Regression modified Features')

Linear Regression modified Features
--------------------------------------------
Accuracy: 0.8455768446799837
Precision:  0.6737930416798905
recall:  0.17036144578313253
f1_score:  0.2715976804571631
--------------------------------------------
